In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///../mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [2]:
client.search_experiments()

[<Experiment: artifact_location='/home/pytholic/mlops-zoomcamp-pytholic/02-experiment-tracking/mlruns/2', creation_time=1703051127573, experiment_id='2', last_update_time=1703051127573, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/home/pytholic/mlops-zoomcamp-pytholic/02-experiment-tracking/mlruns/1', creation_time=1702972736474, experiment_id='1', last_update_time=1702972736474, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [3]:
# client.create_experiment(name="my-cool-experiment")

MlflowException: Experiment(name=my-cool-experiment) already exists. Error: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely)
(sqlite3.IntegrityError) UNIQUE constraint failed: experiments.name
[SQL: INSERT INTO experiments (name, artifact_location, lifecycle_stage, creation_time, last_update_time) VALUES (?, ?, ?, ?, ?)]
[parameters: ('my-cool-experiment', None, 'active', 1703055253719, 1703055253719)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [5]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids="1",
    filter_string="metrics.rmse < 6.5",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [6]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 82f2550f1f434fb19c7bb469e8cbd65b, rmse: 6.3136
run id: 41bb08c4a04846ad887a28f2715144c8, rmse: 6.3136
run id: 877327de065a4c4cb46e98d7a73e77d1, rmse: 6.3136
run id: 2af3493cbf794964a8466b58fe6e7add, rmse: 6.3143
run id: 6c4e006415fd493c821d7b2fccbabc16, rmse: 6.3334


In [7]:
# Add models to registry with python api

import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [27]:
run_id = "82f2550f1f434fb19c7bb469e8cbd65b"
model_uri = f"runs:/{run_id}/model"
# mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

In [16]:
# Transition model from one stage to another
client.search_registered_models()

[<RegisteredModel: aliases={'challenger': 1}, creation_timestamp=1703049215020, description='The NYC taxi predictor for trip duration.', last_updated_timestamp=1703051844231, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1703049215070, current_stage='Staging', description='', last_updated_timestamp=1703050379960, name='nyc-taxi-regressor', run_id='2af3493cbf794964a8466b58fe6e7add', run_link='', source='/home/pytholic/mlops-zoomcamp-pytholic/02-experiment-tracking/mlruns/1/2af3493cbf794964a8466b58fe6e7add/artifacts/models_mlflow', status='READY', status_message=None, tags={'model': 'xgboost'}, user_id=None, version=1>,
  <ModelVersion: aliases=[], creation_timestamp=1703049231972, current_stage='Production', description='', last_updated_timestamp=1703051844231, name='nyc-taxi-regressor', run_id='84107e14a76c4207a33fd1f1becb5ce2', run_link='', source='/home/pytholic/mlops-zoomcamp-pytholic/02-experiment-tracking/mlruns/1/84107e14a76c4207a33fd1f1becb5ce2/artifacts/model',

In [17]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging
version: 2, stage: Production
version: 4, stage: None


/tmp/ipykernel_9686/669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [19]:
model_version = 4
new_stage = "Staging"

# Stage a model
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_9686/3514283674.py:5: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1703051796145, current_stage='Staging', description=None, last_updated_timestamp=1703056673591, name='nyc-taxi-regressor', run_id='82f2550f1f434fb19c7bb469e8cbd65b', run_link=None, source='/home/pytholic/mlops-zoomcamp-pytholic/02-experiment-tracking/mlruns/1/82f2550f1f434fb19c7bb469e8cbd65b/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [23]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1703051796145, current_stage='Staging', description='The model version 4 was transitioned to Staging on 2023-12-20', last_updated_timestamp=1703056778878, name='nyc-taxi-regressor', run_id='82f2550f1f434fb19c7bb469e8cbd65b', run_link=None, source='/home/pytholic/mlops-zoomcamp-pytholic/02-experiment-tracking/mlruns/1/82f2550f1f434fb19c7bb469e8cbd65b/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

## Comparing versions and selecting the new "Production" model
 The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.
 These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.

**Note:** The model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.

In [63]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    # Calculate duration
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    # Filter between 1 min and 60 mins
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']    
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(model_name, model_version, X_test, y_test):
    model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

def test_model_2(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [45]:
df = read_dataframe('../data/green_tripdata_2021-03.parquet')

In [46]:
model_name = "nyc-taxi-regressor"

In [47]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.') 

'/home/pytholic/mlops-zoomcamp-pytholic/02-experiment-tracking/preprocessor'

In [48]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [49]:
X_test = preprocess(df, dv)

In [50]:
target = "duration"
y_test = df[target].values

In [53]:
model_version = 1
%time test_model(model_name=model_name, model_version=model_version, X_test=X_test, y_test=y_test)

/home/pytholic/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [08:01:00] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


CPU times: user 15.4 s, sys: 5.33 ms, total: 15.4 s
Wall time: 4.26 s


{'rmse': 6.269336713208072}

In [54]:
model_version = 2
%time test_model(model_name=model_name, model_version=model_version, X_test=X_test, y_test=y_test)

CPU times: user 39.4 ms, sys: 156 µs, total: 39.6 ms
Wall time: 39.1 ms


{'rmse': 7.720735231402423}

In [55]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Production",
    archive_existing_versions=True
)

/tmp/ipykernel_9686/1316468422.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=['challenger'], creation_timestamp=1703049215070, current_stage='Production', description='', last_updated_timestamp=1703059344592, name='nyc-taxi-regressor', run_id='2af3493cbf794964a8466b58fe6e7add', run_link='', source='/home/pytholic/mlops-zoomcamp-pytholic/02-experiment-tracking/mlruns/1/2af3493cbf794964a8466b58fe6e7add/artifacts/models_mlflow', status='READY', status_message=None, tags={'model': 'xgboost'}, user_id=None, version=1>

In [66]:
%time test_model_2(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

/home/pytholic/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [08:15:18] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


CPU times: user 15.3 s, sys: 0 ns, total: 15.3 s
Wall time: 3.94 s


{'rmse': 6.269336713208072}